# DATA CLEANING APPENDIX

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

## Data for NYC Poverty 2013

In [2]:
#reading in csv file
raw_poverty = pd.read_csv('nycgov_poverty_2013_.csv')

# preserving a raw copy of the poverty data
poverty = raw_poverty.copy()

# dropping all columns except the ones relevant to our data set: the housing unit identifier ('SERIALNO'), Race/Ethinicty ('Ethnicity'), NYC Borough ('BORO'), NYCgov Poverty Status ('NYCgov_POV_Stat')
poverty = poverty[['SERIALNO', 'Ethnicity', 'Boro', 'NYCgov_Pov_Stat']]

# renaming some columns
poverty = poverty.rename({'SERIALNO':'identifier', 'Ethnicity':'ethnicity', 'Boro':'boro', 'NYCgov_Pov_Stat':'povstat'}, axis=1)

# ethnicity filts
filt1 = (poverty['ethnicity'] == 1)
poverty.loc[filt1,'ethnicity'] = 'white'

filt2 = (poverty['ethnicity'] == 2)
poverty.loc[filt2,'ethnicity'] = 'black'

filt3 = (poverty['ethnicity'] == 3)
poverty.loc[filt3,'ethnicity'] = 'asian'

filt4 = (poverty['ethnicity'] == 4)
poverty.loc[filt4,'ethnicity'] = 'hispanic'

filt5 = (poverty['ethnicity'] == 5)
poverty.loc[filt5,'ethnicity'] = 'other'

#boro filts
filt6 = (poverty['boro'] == 1)
poverty.loc[filt6,'boro'] = 'Bronx'

filt7 = (poverty['boro'] == 2)
poverty.loc[filt7,'boro'] = 'Brooklyn'

filt8 = (poverty['boro'] == 3)
poverty.loc[filt8,'boro'] = 'Manhattan'

filt9 = (poverty['boro'] == 4)
poverty.loc[filt9,'boro'] = 'Queens'

filt10 = (poverty['boro'] == 5)
poverty.loc[filt10,'boro'] = 'Staten Island'


#change the 0 to mean to mean not in poverty as it was originally 2
filt = (poverty['povstat'] == 2)
poverty.loc[filt,'povstat'] = 0

# poverty.head()


### Explaining what each column means for the ORIGINAL poverty data (This has been altered):

The SERIALNO column is a housing unit identifier determined by the serial number of each Census household in the survey. 

This is the key for each number in the Ethnicity column: 
* 1 Non-Hispanic White Individual ACS 
* 2 Non-Hispanic Black 
* 3 Non-Hispanic Asian 
* 4 Hispanic, Any Race 
* 5 Other Race/Ethnic Group

This is the key for each number in the Boro column:
* 1 Bronx  
* 2 Brooklyn 
* 3 Manhattan 
* 4 Queens 
* 5 Staten Island

The NYCgov_Pov_Stat column defines whether a respondent is in poverty (1) or not in poverty (2)

## HIV Testing Locations

In [3]:
#reading testing location csv
hiv_test_loc= pd.read_csv('hiv_testing_locations.csv')

#changing all the boroughs to upper sincee QUEENS vs Queens appears in dataset
hiv_test_loc ["Borough"] = hiv_test_loc["Borough"].str.upper()

#dropping columns like hours etc. If interested look at the raw csv on Github.
clean_hiv_loc = hiv_test_loc[['Site Name', 'Borough', 'Zip Code', 'Low Cost', 'Free', 'Address']]
## renaming the columns
clean_hiv_loc = clean_hiv_loc.copy()
clean_hiv_loc.rename(columns = {'Site Name':'site_name', 'Borough': 'borough', 'Zip Code': 'zip_code', 'Low Cost': 'low_cost','Free':'free', 'Address':'address'}, inplace = True) 

## this is the cleaned dataset with values dropped and grouped
droppedna_cleaned = clean_hiv_loc.copy()
droppedna_cleaned = droppedna_cleaned.dropna(subset=['zip_code'])
droppedna_cleaned = droppedna_cleaned.dropna(subset=['address'])
droppedna_cleaned = droppedna_cleaned.dropna(subset=['borough'])

# Changing the borough from New York to Manhattan
droppedna_cleaned.at[521, 'borough'] = 'MANHATTAN'
droppedna_cleaned.head(6)




,site_name,borough,zip_code,low_cost,free,address
0,Office of the Manhattan Borough President,MANHATTAN,10007,False,False,1 Centre Street
1,Clinic - Lenox Avenue,MANHATTAN,10026,True,True,115 West 116th Street
2,HHC Gouverneur Health,MANHATTAN,10002,True,True,227 Madison Street
3,Morrisania Chest Center Clinic,BRONX,10456,False,True,1309 Fulton Avenue
4,BronxCare Fulton Family Medicine,BRONX,10456,False,False,1276 Fulton Avenue
5,Community Health Action of Staten Island- Main...,STATEN ISLAND,10301,False,False,56 Bay Street


## HIV Diagnoses by Neighborhood

In [4]:
raw_hiv = pd.read_csv('hiv_aids_diagnoses.csv')

new_cols = [name.lower() for name in raw_hiv.columns]
new_cols = [name.replace(' ','_') for name in new_cols]
raw_hiv.columns = new_cols

# making a new copy of the data set to preserve a raw copy
hiv = raw_hiv.copy()

# dropped columns that are irrelevant to our research question
hiv = hiv.drop(columns=['hiv_diagnoses_per_100,000_population', 
                        'total_number_of_concurrent_hiv/aids_diagnoses', 
                        'proportion_of_concurrent_hiv/aids_diagnoses_among_all_hiv_diagnoses', 'aids_diagnoses_per_100,000_population'])

# changing the name of the UHF neighborhood column to make it easier to work with
hiv = hiv.rename({'neighborhood_(u.h.f)':'uhf_neighborhood', 'total_number_of_hiv_diagnoses':'total_hiv', 'total_number_of_aids_diagnoses':'total_aids'}, axis=1)
hiv.head()

# dropping rows that have their neighborhood listed as 'unknown'
hiv = hiv[hiv['uhf_neighborhood'] != 'Unknown']
# hiv.astype({'total_hiv': 'int64'}).dtypes
hiv = hiv[hiv['total_hiv'] != '*']
# print(hiv.describe())
# hiv.head()

In [5]:
 #replace the asterick because it was causing me problems
hiv.replace('*', -1, inplace= True)

#casting the object type column to int so I can sum them together
hiv['total_aids'] = hiv['total_aids'].astype(int)
hiv['total_hiv'] = hiv['total_hiv'].astype(int)
hiv['year'] = hiv['year'].astype(str)
hiv['uhf_neighborhood'] = hiv['uhf_neighborhood'].astype(str)

#creating filters for dropping all the -1 
drop_index_h = hiv[ hiv['total_hiv'] == -1 ].index
drop_index_a = hiv[ hiv['total_aids'] == -1 ].index

#dropping by using filters
hiv.drop(drop_index_h, inplace = True)
hiv.drop(drop_index_a, inplace = True)

##making sure that it worked
# grouped_hiv = hiv.groupby('year').total_hiv.sum()
# grouped_hiv.head()

## Condom Availability
#### NOTE THIS DATA IS FROM MARCH 17th 2021 as the data is renewed DAILY

In [6]:
ori_condom = pd.read_csv('nyc_condom_availability.csv')

# preserving a copy of the raw condom availability data
condom = ori_condom.copy()

# only keeping columns that are relevant to our research question
condom = condom[['Facility PK', 'FacilityName', 'Address', 'Borough', 'Zipcode', 'Condoms (Male)', 'FC2 (Female/Insertive Condoms)']]

#renaming columns so they're easier to work with
condom = condom.rename({'Facility PK':'identifier', 'FacilityName':'name', 'Address':'address', 'Borough':'boro', 'Zipcode':'zip', 'Condoms (Male)':'malecond', 'FC2 (Female/Insertive Condoms)':'femalecond'}, axis=1)

# replacing the NaN for the borough of the 'C/O NYC FREE CONDOM' distribution site with the proper borough
condom.loc[0, 'boro'] = 'Queens'

# adding a proper zipcode for the sites that did not provide one
condom.loc[condom['identifier'] == 113039632, 'zip'] = 10030
condom.loc[condom['identifier'] ==113039503, 'zip'] = 10039
condom.loc[condom['identifier']==113039855, 'zip'] = 11226
condom.loc[condom['identifier']==113039591,'zip'] = 10026
condom.loc[condom['identifier']==113039689, 'zip'] = 10024
condom.loc[condom['identifier']==113039479, 'zip'] = 10030
condom.loc[condom['identifier']==113039590, 'zip'] = 10039
condom.loc[condom['identifier']== 113039585, 'zip'] = 11207
condom.loc[condom['identifier']==113039612, 'zip'] = 10027

# condom.head()

### Explaining columns for the condom availability data:
Facility PK is the autogenerated facility identification key from the Department of Health and Mental Hygiene.

FacilityName is the name of the service facility.

Address, Borough and Zipcode describe the location of the service facility.

Condoms (Male) describes whether male columns are available at the service facility, and FC2 (Female/Insertive Condoms) describes whether female columns are available at the facility

### NOTE: More data manipulation will be done in request/function calls to API when acquring geodata in final.ipynb
- The decision was made to maintain the current data frame as the address are valid and current via Google Maps but Folium is less advance (but free) so modifications to address were made during the function call. 
    - The code modifiying locations from final.ipynb can be used to create a new dataframe and loop over that if desired.